<a href="https://colab.research.google.com/github/SheethalVelutharambath/citation_intent/blob/main/without_pos_traditional_classsifers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import gensim
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [ ]:

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/daaaataaa_citae/tsv/train.tsv", sep= '\t',names=["id", "explicit", "text", "label"])
dev = pd.read_csv("/content/drive/MyDrive/daaaataaa_citae/tsv/dev.tsv", sep= '\t')
test = pd.read_csv("/content/drive/MyDrive/daaaataaa_citae/tsv/test.tsv", sep= '\t', names=["id", "explicit", "text", "label"])

In [ ]:
train.drop(train.columns[[0, 1]], axis = 1, inplace = True)
test.drop(test.columns[[0, 1]], axis = 1, inplace = True)

In [ ]:
train.head()

,text,label
0,"However, how frataxin interacts with the Fe-S ...",background
1,"In the study by Hickey et al. (2012), spikes w...",background
2,"The drug also reduces catecholamine secretion,...",background
3,By clustering with lowly aggressive close kin ...,background
4,Ophthalmic symptoms are rare manifestations of...,background


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
train['text1']= train['text'].apply(lambda x:remove_punctuation(x))
test['text'] = test['text'].apply(lambda x:remove_punctuation(x))
train['labels'] = train['label'].replace(['background','method','result'],[0,1,2])
test['label'] = test['label'].replace(['background','method','result'],[0,1,2])
test.head()

,text,label
0,Chapel as well as X10 2 UPC 3 CoArray Fortran...,0
1,In addition the result of the present study su...,2
2,Several instruments that more specifically add...,0
3,Organotypic hippocampal slice culturesnInterfa...,1
4,Activated PBMC are the basis of the standard P...,0


In [ ]:
train['text1']= train['text1'].apply(lambda x: x.lower())
test['text'] = test['text'].apply(lambda x: x.lower())

In [ ]:
train.head()

,text,label,text1,labels
0,"However, how frataxin interacts with the Fe-S ...",background,however how frataxin interacts with the fes cl...,0
1,"In the study by Hickey et al. (2012), spikes w...",background,in the study by hickey et al 2012 spikes were ...,0
2,"The drug also reduces catecholamine secretion,...",background,the drug also reduces catecholamine secretion ...,0
3,By clustering with lowly aggressive close kin ...,background,by clustering with lowly aggressive close kin ...,0
4,Ophthalmic symptoms are rare manifestations of...,background,ophthalmic symptoms are rare manifestations of...,0


In [ ]:
test.head()

,text,label
0,chapel as well as x10 2 upc 3 coarray fortran...,0
1,in addition the result of the present study su...,2
2,several instruments that more specifically add...,0
3,organotypic hippocampal slice culturesninterfa...,1
4,activated pbmc are the basis of the standard p...,0


In [ ]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def tokenize(word):
   word = nltk.word_tokenize(word)
   return word

In [ ]:
train['tokenized']= train['text1'].apply(lambda x: tokenize(x))

In [ ]:
test['tokenized']= test['text'].apply(lambda x: tokenize(x))

In [ ]:
def remove_stopwords(texts):
    output= [i for i in texts if i not in stopwords]
    return output

In [ ]:
train['tokenized']= train['tokenized'].apply(lambda x:remove_stopwords(x))

In [ ]:
test['tokenized']= test['tokenized'].apply(lambda x:remove_stopwords(x))

In [ ]:
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text
train['stemmed']=train['tokenized'].apply(lambda x:lemmatizer(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
test['stemmed']=test['tokenized'].apply(lambda x:lemmatizer(x))

In [ ]:
train.head()

,text,label,text1,labels,tokenized,stemmed
0,"However, how frataxin interacts with the Fe-S ...",background,however how frataxin interacts with the fes cl...,0,"[however, frataxin, interacts, fes, cluster, b...","[however, frataxin, interacts, fe, cluster, bi..."
1,"In the study by Hickey et al. (2012), spikes w...",background,in the study by hickey et al 2012 spikes were ...,0,"[study, hickey, et, al, 2012, spikes, sampled,...","[study, hickey, et, al, 2012, spike, sampled, ..."
2,"The drug also reduces catecholamine secretion,...",background,the drug also reduces catecholamine secretion ...,0,"[drug, also, reduces, catecholamine, secretion...","[drug, also, reduces, catecholamine, secretion..."
3,By clustering with lowly aggressive close kin ...,background,by clustering with lowly aggressive close kin ...,0,"[clustering, lowly, aggressive, close, kin, ki...","[clustering, lowly, aggressive, close, kin, ki..."
4,Ophthalmic symptoms are rare manifestations of...,background,ophthalmic symptoms are rare manifestations of...,0,"[ophthalmic, symptoms, rare, manifestations, i...","[ophthalmic, symptom, rare, manifestation, int..."


In [ ]:
x_test = test.text.tolist()

In [ ]:
test.head()

,text,label,tokenized,stemmed
0,chapel as well as x10 2 upc 3 coarray fortran...,0,"[chapel, well, x10, 2, upc, 3, coarray, fortra...","[chapel, well, x10, 2, upc, 3, coarray, fortra..."
1,in addition the result of the present study su...,2,"[addition, result, present, study, supports, p...","[addition, result, present, study, support, pr..."
2,several instruments that more specifically add...,0,"[several, instruments, specifically, address, ...","[several, instrument, specifically, address, p..."
3,organotypic hippocampal slice culturesninterfa...,1,"[organotypic, hippocampal, slice, culturesnint...","[organotypic, hippocampal, slice, culturesnint..."
4,activated pbmc are the basis of the standard p...,0,"[activated, pbmc, basis, standard, pbmc, blast...","[activated, pbmc, basis, standard, pbmc, blast..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
td = TfidfVectorizer(max_features = 4500)
X = td.fit_transform(train['text1']).toarray()

In [ ]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
td1 = TfidfVectorizer(max_features = 4500)
X_test = td1.fit_transform(test['text']).toarray()

In [ ]:
y = train['labels'].tolist()

In [ ]:
Y_test = test['label'].tolist()

In [ ]:
from sklearn.linear_model import SGDClassifier
lr = SGDClassifier(loss='log', penalty='l1')
lr.fit(X, y)
preds = lr.predict(X_test)
print(classification_report(Y_test, preds))

              precision    recall  f1-score   support

           0       0.54      0.98      0.69       997
           1       0.42      0.02      0.04       605
           2       0.40      0.01      0.02       259

    accuracy                           0.53      1861
   macro avg       0.45      0.34      0.25      1861
weighted avg       0.48      0.53      0.39      1861



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X, y)
predictions = knn.predict(X_test)


print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.54      0.75      0.63       997
           1       0.36      0.24      0.28       605
           2       0.14      0.03      0.06       259

    accuracy                           0.48      1861
   macro avg       0.34      0.34      0.32      1861
weighted avg       0.42      0.48      0.44      1861



In [ ]:
from sklearn.datasets import load_digits
from sklearn.linear_model import Perceptron
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X,y)
y_pred = clf.predict(X_test)

print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.84      0.66       997
           1       0.35      0.18      0.24       605
           2       0.00      0.00      0.00       259

    accuracy                           0.51      1861
   macro avg       0.30      0.34      0.30      1861
weighted avg       0.40      0.51      0.43      1861



In [ ]:
mod = SVC(kernel = 'linear', C=1)
mod.fit(X,y)
y_preds = clf.predict(X_test)

print(classification_report(Y_test, y_preds))

              precision    recall  f1-score   support

           0       0.54      0.84      0.66       997
           1       0.35      0.18      0.24       605
           2       0.00      0.00      0.00       259

    accuracy                           0.51      1861
   macro avg       0.30      0.34      0.30      1861
weighted avg       0.40      0.51      0.43      1861

